<span style="font-size:150%">라이브러리</span>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

<span style="font-size:150%">경로 및 파라미터</span>

In [ ]:
lecture_root = os.path.dirname(os.getcwd())
data_path = os.path.join(lecture_root, 'data', 'cifar10')
ckp_path = os.path.join(lecture_root, 'checkpoints', 'classification')
os.makedirs(data_path, exist_ok=True)
os.makedirs(ckp_path, exist_ok=True)

num_epoch = 10
batch_size = 64
learning_rate = 0.001

device = 'cuda' if torch.cuda.is_available() else 'cpu'


<span style="font-size:150%">어그멘테이션</span>

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

test_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

<span style="font-size:150%">데이터셋</span>

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10(
    root=data_path,
    train=True,
    download=True,
    transform=train_transform
)
test_data = datasets.CIFAR10(
    root=data_path,
    train=False,
    download=False,
    transform=test_transform
)

<span style="font-size:150%">데이터로더</span>

In [ ]:
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

<span style="font-size:150%">모델 설계</span>

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Linear(32*8*8, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        flatten = out.view(out.size(0), -1)
        score = self.fc(flatten)
        return score

<span style="font-size:150%">모델 및 손실함수, 옵티마이저 선언</span>

In [ ]:
mode = 1

if mode==1:
    model = ConvNet()
elif mode==2:
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(512, 10)
elif mode==3:
    from torchvision.models import resnet18, ResNet18_Weights
    model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(512, 10)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

<span style="font-size:150%">모델 학습 함수</span>

In [ ]:
def train_loop():
    # model train mode로 전환
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    for (inputs, labels) in tqdm(trainloader, 'Train'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, pred = torch.max(outputs, 1)
        total += outputs.size(0)
        running_acc += (pred == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    total_loss = running_loss / len(trainloader)
    total_acc = 100 * running_acc / total
    print(f'Train loss : {total_loss} Acc : {total_acc}%')

<span style="font-size:150%">모델 평가 함수</span>

In [ ]:
def test_loop():
    # model eval mode로 전환
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    with torch.no_grad():
        for (inputs, labels) in tqdm(testloader, "Test"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, pred = torch.max(outputs, 1)
            total += outputs.size(0)
            running_acc += (pred == labels).sum().item()
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        total_loss = running_loss / len(testloader)
        total_acc = 100 * running_acc / total
        print(f'Test loss : {total_loss} Acc : {total_acc}%')

<span style="font-size:150%">모델 학습 및 평가</span>

In [ ]:
for epoch in range(num_epoch):
    print(f'Epoch {epoch}')
    train_loop()
    test_loop()
    path = os.path.join(ckp_path, f'classifier_{epoch}.pth')
    torch.save(model.state_dict(), path)